In [5]:
import time
from collections import defaultdict
import pandas as pd
from collections import OrderedDict
from datetime import date
import string
import nltk
from nltk.corpus import stopwords
# from tweepy import Stream
# from tweepy import OAuthHandler
# from tweepy.streaming import StreamListener
import json
import tweepy
from nltk.tokenize import TweetTokenizer

# Load in Tweets saved from extract process

In [2]:
# Load in all of the extract files into separate data frames
df = pd.read_csv('C:\data\CS5664\IPA_Teets_ALL.csv')
print "Number of Tweets: " + str(df["id"].size)

Number of Tweets: 5951


In [7]:
raw_tweets = df['text'].values
exclude = set(string.punctuation)
tokenized_tweets = []
tokenizer = TweetTokenizer()
for tweet_text in raw_tweets:
    tokens = tokenizer.tokenize(tweet_text.lower())
    tokenized_tweets.append(''.join([ch for ch in ' '.join(tokens) if ch not in exclude]).split())
print(tokenized_tweets[3])

[u'victortellegio', u'jayho79', u'not', u'to', u'hijack', u'but', u'maui', u'brewings', u'double', u'blood', u'orange', u'ipa', u'in', u'the', u'can', u'is', u'legit', u'as', u'hell', u'\ud83d', u'\udc4c', u'\ud83c', u'\udffb']


In [90]:
sws = set(stopwords.words('english'))
sws.add('rt') # Tweet specific stop-words
sws.add(u'https\u2026') # Tweet specific stop-words
sws.add(u'\u2026') 
sws.add(u'\udea8')
sws.add(u'\ud83d')
sws.add(u'\u201c')
sws.add(u'\udc4c')
sws.add(u'\ud83c')
sws.add(u'\udffb')
sws.add(u'�')
sws.add(u'�')
sws.add(u'—')
sws.add(u'’')
sws.add(u'�')
sws_removed_tweets = []
for j,sent in enumerate(tokenized_tweets):
    sws_removed_tweets.append(([i for i in sent if i not in sws]))

    
#        print "Tweet: " + str(i) + " Sent: " + str(j) + " Without - ", ' '.join([i for i in sent.lower().split() if i not in sws])
    
print raw_tweets[1]
print sws_removed_tweets[1]

I just earned the 'I Believe in IPA! (Level 4)' badge on @untappd! https://t.co/Hz8NQ0owBi #ibelieveinIPA
[u'earned', u'believe', u'ipa', u'level', u'4', u'badge', u'untappd', u'httpstcohz8nq0owbi', u'ibelieveinipa']


In [83]:
flattened_words = [item for sublist in sws_removed_tweets for item in sublist]
df_words = pd.DataFrame(flattened_words)
print df_words.size

67240


In [85]:
print df_words[0].value_counts()[0:10]

ipa         4431
drinking    1278
untappd      630
ale          620
level        596
badge        589
earned       587
believe      586
pale         584
india        518
Name: 0, dtype: int64


## Topic Generation

In [69]:
from gensim import corpora, models
dictionary = corpora.Dictionary(sws_removed_tweets)

C:\Anaconda2\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [72]:
corpus = [dictionary.doc2bow(text) for text in sws_removed_tweets]

In [73]:
print(corpus[2])

[(4, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)]


In [74]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [78]:
res = ldamodel.print_topics(num_topics=3, num_words=3)
for r in res:
    print u'Topic ' + str(r[0]) + ": " + r[1]

Topic 0: 0.031*"ipa" + 0.008*"de" + 0.006*"la"
Topic 1: 0.060*"ipa" + 0.033*"drinking" + 0.019*"—"
Topic 2: 0.062*"ipa" + 0.025*"untappd" + 0.024*"level"


## Entity Extraction

In [86]:
from collections import defaultdict
def nltk_entities(text):
    """
    Extract entities using the NLTK named entity chunker.
    """
    text = nltk.pos_tag(nltk.word_tokenize(text))
    results = defaultdict(list)
    lst_results = []
    for entity in nltk.ne_chunk(text):
        if isinstance(entity, nltk.tree.Tree):
            etext = " ".join([word for word, tag in entity.leaves()])
            label = entity.label()
        else:
            continue

        if label == 'PERSON':
            key = 'persons'
        elif label == 'ORGANIZATION':
            key = 'organizations'
        elif label == 'LOCATION':
            key = 'locations'
        elif label == 'GPE':
            key = 'other'
        else:
            key = None

        if key:
            results[key].append(etext)
            lst_results.append(etext)

    return lst_results

In [103]:
entity_list = []
import unicodedata
# look at the entities for the first few tweets
for tweet in raw_tweets:
    ntweet = unicodedata.normalize('NFKD', tweet.decode('unicode-escape')).encode('ascii','ignore')
#    print nltk_entities(ntweet)
    entity_list.append(nltk_entities(ntweet))

In [104]:
ind_entities = [item for sublist in entity_list for item in sublist]
df_ind_entities = pd.DataFrame(ind_entities)
print df_ind_entities.size

11501


In [114]:
print df_ind_entities[0].value_counts()[100:200]

Robotics                    10
JmarrMarr                   10
Google                      10
Public                      10
Hazy                        10
Pilsner                     10
Cascade                     10
IPa                          9
Race Commissionera           9
SierraNevada                 9
SenatorFifield               9
PwC                          9
Amager Bryghus               9
UK                           9
Brewery                      9
English                      9
BrewDog                      9
Voodoo Ranger Juicy Haze     9
East Coast                   9
BellaFlokarti                9
Craft                        9
IntPublishers                9
Little Johnny Howard         9
Juicy                        9
Session                      9
Huell Melon                  8
Stone                        8
Cheers                       8
France                       8
APS Commissioner             8
                            ..
Nancy Bird                   7
Union Ja